In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [4]:
hflight = spark.read.option('inferSchema', 'true').\
    option('header', 'true').\
    csv("file:///home/hadoop/jupyter/dataengineering/dataset/hflight.csv")

In [13]:
hflight.count()

227496

## Pyspark 버전
### 출발 공항에 대해서 도착 공항별로 평균 출발 지연, 평균 도착 지연시간을 구하기!
#### groupby
#### avg,mean

In [14]:
hflight.limit(1).show()

+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|ArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|
+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|2011|    1|         1|        6|   1400|   1500|           AA|      428| N576AA|               60|     40|     -10|       0|   IAH| DFW|     224|     7|     13|        0|            null|       0|
+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+



In [15]:
help(hflight)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(pyspark.sql.pandas.map_ops.PandasMapOpsMixin, pyspark.sql.pandas.conversion.PandasConversionMixin)
 |  DataFrame(jdf: py4j.java_gateway.JavaObject, sql_ctx: Union[ForwardRef('SQLContext'), ForwardRef('SparkSession')])
 |
 |  A distributed collection of data grouped into named columns.
 |
 |  .. versionadded:: 1.3.0
 |
 |  .. versionchanged:: 3.4.0
 |      Supports Spark Connect.
 |
 |  Examples
 |  --------
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`:
 |
 |  >>> people = spark.createDataFrame([
 |  ...     {"deptId": 1, "age": 40, "name": "Hyukjin Kwon", "gender": "M", "salary": 50},
 |  ...     {"deptId": 1, "age": 50, "name": "Takuya Ueshin", "gender": "M", "salary": 100},
 |  ...     {"deptId": 2, "age": 60, "name": "Xinrong Meng", "gender": "F", "salary": 150},
 |  ...     {"deptId": 3, "age": 20, "name": "

In [ ]:
hflight.printSchema()

In [5]:
from pyspark.sql.functions import avg, stddev, col

hflight = hflight.withColumn('ArrDelay', col('ArrDelay').cast('integer')).\
            withColumn('DepDelay', col('DepDelay').cast('integer'))

#hflight.groupby("Origin", "Dest").mean("ArrDelay", "DepDelay").show()
hflight.groupBy("Origin", "Dest").agg(
    avg("ArrDelay").alias("출발 지연 평균"),
    avg("DepDelay").alias("도착 지연 평균")).show()

+------+----+------------------+------------------+
|Origin|Dest|    출발 지연 평균|    도착 지연 평균|
+------+----+------------------+------------------+
|   HOU| PHL|23.541666666666668| 33.19124423963134|
|   IAH| LIT| 6.764063811922754| 7.740771812080537|
|   IAH| GSP| 9.617359413202934|  9.08272506082725|
|   IAH| DEN| 7.441828254847645| 10.37734288864388|
|   IAH| TUS| 7.801679586563307| 7.783870967741936|
|   IAH| ABQ| 7.928692699490663| 6.703891708967851|
|   HOU| LIT| 3.533333333333333|7.4404432132963985|
|   IAH| ANC|26.080645161290324|            24.952|
|   IAH| SMF|  4.66271018793274| 9.093966369930762|
|   HOU| TUL| 7.855238095238096|11.208174904942966|
|   HOU| MDW|3.8126815101645692|13.204150579150578|
|   HOU| OKC| 9.441897233201582|12.509865824782953|
|   IAH| DSM|15.951104100946372|12.796850393700787|
|   IAH| DFW| 5.510741008930727| 8.764620938628159|
|   IAH| BOS| 6.559045956951716|10.519141531322505|
|   HOU| MAF| 6.918367346938775|10.507042253521126|
|   IAH| RIC|12.30123180

# 문제 2
#### 목적지 공항에 대해 연착 건수를 구하고,
#### 연착 건수가 2000회 이상인 공항에 대한 데이터만 추출
#### col -> Dest : 목적지 공항 ArrDelay(연착은 5분이상)

In [57]:
help(hflight.sort)

Help on method sort in module pyspark.sql.dataframe:

sort(*cols: Union[str, pyspark.sql.column.Column, List[Union[str, pyspark.sql.column.Column]]], **kwargs: Any) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` sorted by the specified column(s).

    .. versionadded:: 1.3.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Parameters
    ----------
    cols : str, list, or :class:`Column`, optional
         list of :class:`Column` or column names to sort by.

    Other Parameters
    ----------------
    ascending : bool or list, optional, default True
        boolean or list of boolean.
        Sort ascending vs. descending. Specify list for multiple sort orders.
        If a list is specified, the length of the list must equal the length of the `cols`.

    Returns
    -------
    :class:`DataFrame`
        Sorted DataFrame.

    Examples
    --------
    >>> from pyspark.sql.functions import desc, asc
    >>>

In [59]:
from pyspark.sql.functions import count, col
hflight= hflight.withColumn('ArrDelay', col('ArrDelay').cast('integer'))
hflight.filter(col('ArrDelay') > 5).groupby("Dest").agg(count(col("ArrDelay")).alias("Cnt")).filter(col("Cnt") > 2000).sort("Cnt", ascending=False).show()

"""
from pyspark.sql.functions import expr
hflight.withColumn("ArrDelay", expr("ArrDelay == 0")).show()
hflight.where(expr("ArrDelay == 0")).show()
"""

+----+----+
|Dest| Cnt|
+----+----+
| DAL|3160|
| ATL|2515|
| LAX|2492|
| MSY|2201|
| DEN|2177|
| ORD|2118|
+----+----+



'\nfrom pyspark.sql.functions import expr\nhflight.withColumn("ArrDelay", expr("ArrDelay == 0")).show()\nhflight.where(expr("ArrDelay == 0")).show()\n'

In [39]:
from pyspark.sql.functions import expr
hflight.withColumn("ArrDelay", expr("ArrDelay == 0")).show()
hflight.where(expr("ArrDelay == 0")).show()

target = hflight.withColumn("count", expr("ArrDelay >=5")).\
    where(expr("count == True")).\
    groupby("Dest").\
    count().\
    filter(expr("count >= 2000")).select("Dest").rdd.flatMap(lambda x : x).collect()


hflight2 = hflight.where(col('Dest').isin(target))

hflight2.select("Dest").distinct().show()

+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|ArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|
+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|2011|    1|         1|        6|   1400|   1500|           AA|      428| N576AA|               60|     40|   false|       0|   IAH| DFW|     224|     7|     13|        0|            null|       0|
|2011|    1|         2|        7|   1401|   1501|           AA|      428| N557AA|               60|     45|   false|       1|   IAH| DFW|     224|     6|      9|        0|            null|       0|
|2011|    

In [2]:
### JOIN
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [3]:
empDF.join( deptDF, empDF.emp_dept_id == deptDF.dept_id, "inner").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



## 문제 3
### 위의 결과를 바탕으로 목적지  공항 별 경항횟수, 회항횟수
### 운항 횟수를 구하시오
### 운항 횟수는 결항과 회항을 제외한 것

## 문제3 해설

In [48]:
hflight4 = spark.read.parquet("hdfs://namenode:8020/20240619/scw/hflight_ex3.parquet")

In [65]:
hflight4.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)



In [69]:
from pyspark.sql.functions import expr
url = "jdbc:mysql://43.202.5.70:3306/encore_web"
properties = {
    "user": "class5",
    "password": 'EnCoRo!23',
    "driver": "com.mysql.cj.jdbc.Driver"
}

a = hflight4.groupBy("Dest").sum("Cancelled")
b = hflight4.groupBy("Dest").sum("Diverted")
c = hflight4.groupBy("Dest").count()

result = a.join(b, on="Dest", how='left').\
    join(c, on="Dest", how='left').\
    withColumnRenamed("sum(Cancelled)", "Cancelled").\
    withColumnRenamed("sum(Diverted)", "Diverted").\
    withColumn("air", expr("count - Cancelled - Diverted")).\
    drop("count").\
    write.jdbc(url=url, table='hiyo', mode='append', properties=properties)


# 하둡 연동

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


df = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://43.202.5.70:3306/encore_web") \
    .option("dbtable","encore_question") \
    .option("numPartitions",5) \
    .option("user", "class5") \
    .option("password", 'EnCoRo!23') \
    .load()



### df = spark.read **:

df는 우리가 데이터를 저장할 변수의 이름
spark.read는 데이터를 읽어오는 시작점

### .format("jdbc") **:

jdbc는 Java Database Connectivity의 줄임말로, 데이터베이스에 연결하는 방법
이 코드는 MySQL 데이터베이스에서 데이터를 읽어올 수 있음

### .option("driver","com.mysql.cj.jdbc.Driver") **:

driver는 MySQL 데이터베이스에 연결할 때 사용하는 특별한 프로그램
com.mysql.cj.jdbc.Driver는 MySQL에 연결할 수 있게 해주는 드라이버의 이름

### .option("url", "jdbc:mysql://43.202.5.70:3306/encore_web") **:

url은 우리가 연결할 MySQL 데이터베이스의 주소
jdbc:mysql://43.202.5.70:3306/encore_web는 "43.202.5.70"이라는 컴퓨터에 있는 "encore_web"라는 데이터베이스를 의미

### .option("dbtable","encore_question") **:

dbtable은 데이터베이스 안에 있는 테이블 이름
encore_question 테이블에서 데이터를 읽어옴

### .option("numPartitions",5) **:

numPartitions는 데이터를 몇 개의 조각으로 나눌지 정함
데이터를 5개의 조각으로 나눌 거예요. 이렇게 하면 여러 컴퓨터가 동시에 데이터를 읽을 수 있음

### .option("user", "class5") **:

user는 데이터베이스에 접속할 때 사용하는 사용자 이름
여기서는 "class5"라는 사용자를 사용함

### .option("password", 'EnCoRo!23') **:

password는 데이터베이스에 접속할 때 사용하는 비밀번호
여기서는 'EnCoRo!23'이라는 비밀번호를 사용함

### .load():

load()는 지금까지 설정한 모든 정보를 사용해서 실제로 데이터를 읽어오는 명령
이렇게 하면 df라는 변수에 데이터베이스에서 읽어온 데이터가 저장됨


In [2]:
df.show()

+---+------------------------+-----------------------+--------------------+---------+
| id|                 subject|                content|         create_date|quthor_id|
+---+------------------------+-----------------------+--------------------+---------+
|  1|                       d|                      d|2024-05-14 06:37:...|        1|
|  2|                       d|                      d|2024-05-14 06:37:...|        1|
|  3|                       d|                      d|2024-05-14 06:37:...|        1|
|  4|                 sdfasdf|                 fasadf|2024-05-14 06:38:...|        1|
|  5|                    우와|                   우와|2024-05-14 06:38:...|        1|
|  6|                 sdfasdf|                 fasadf|2024-05-14 06:38:...|        1|
|  8|                    으오|                   ㅇㅇ|2024-05-14 06:39:...|        1|
|  9|    첫글을 빼앗겼네요...|                   ㅠㅠ|2024-05-14 06:39:...|        1|
| 10|                    집에|          가고싶어요...|2024-05-14 06:39:...|    

In [4]:
from pyarrow import fs
import pyarrow as pa
import subprocess

classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='192.168.0.160', port=8020, user='hadoop')

In [5]:
from pyspark.sql.functions import lit 

In [9]:
df2 = df.withColumn("dummy", lit("하하"))
df2.show()

+---+------------------------+-----------------------+--------------------+---------+-----+
| id|                 subject|                content|         create_date|quthor_id|dummy|
+---+------------------------+-----------------------+--------------------+---------+-----+
|  1|                       d|                      d|2024-05-14 06:37:...|        1| 하하|
|  2|                       d|                      d|2024-05-14 06:37:...|        1| 하하|
|  3|                       d|                      d|2024-05-14 06:37:...|        1| 하하|
|  4|                 sdfasdf|                 fasadf|2024-05-14 06:38:...|        1| 하하|
|  5|                    우와|                   우와|2024-05-14 06:38:...|        1| 하하|
|  6|                 sdfasdf|                 fasadf|2024-05-14 06:38:...|        1| 하하|
|  8|                    으오|                   ㅇㅇ|2024-05-14 06:39:...|        1| 하하|
|  9|    첫글을 빼앗겼네요...|                   ㅠㅠ|2024-05-14 06:39:...|        1| 하하|
| 10|                 

In [60]:
df2.write.mode("overwrite").csv("hdfs://namenode:8020/20240619/scw/my3.csv")

In [52]:
df3 = df2.withColumn("dummy2", lit("맨유는 개못해"))

In [53]:
# 해당 경로에 폴더가 없어도 생성해줌
df3.write.parquet("hdfs://namenode:8020/20240619/maenchester/liverpool.parquet")

In [54]:
df4= spark.read.parquet("hdfs://namenode:8020/20240619/maenchester/liverpool.parquet")

In [55]:
df4.show()

+---+------------------------+-----------------------+--------------------+---------+-----+-------------+
| id|                 subject|                content|         create_date|quthor_id|dummy|       dummy2|
+---+------------------------+-----------------------+--------------------+---------+-----+-------------+
|  1|                       d|                      d|2024-05-14 06:37:...|        1| 하하|맨유는 개못해|
|  2|                       d|                      d|2024-05-14 06:37:...|        1| 하하|맨유는 개못해|
|  3|                       d|                      d|2024-05-14 06:37:...|        1| 하하|맨유는 개못해|
|  4|                 sdfasdf|                 fasadf|2024-05-14 06:38:...|        1| 하하|맨유는 개못해|
|  5|                    우와|                   우와|2024-05-14 06:38:...|        1| 하하|맨유는 개못해|
|  6|                 sdfasdf|                 fasadf|2024-05-14 06:38:...|        1| 하하|맨유는 개못해|
|  8|                    으오|                   ㅇㅇ|2024-05-14 06:39:...|        1| 하하|맨유는 개못해|
|  9